In [65]:
import pandas as pd
import requests
from datetime import date
import plotly.figure_factory as ff
import plotly.express as px
import numpy as np

In [2]:
def get_comtrade_data(endpoint_url):
     request = requests.get(url=endpoint_url)
     print (request)
     print ("length is " + str(len(request.json()["dataset"])))
     return (pd.json_normalize(request.json()["dataset"]))

In [21]:
# Endpoints
country_url = "https://comtrade.un.org/Data/cache/reporterAreas.json"
full_com_url = "https://comtrade.un.org/Data/cache/classificationHS.json"
two_com_url = "https://raw.githubusercontent.com/cameronkruse/food-security/main/assets/data/twodigit_hs_class.json"


In [9]:
# read json file into pandas data frame
country_request = requests.get(url=country_url)
country_request.encoding='utf-8-sig'
country_df = pd.json_normalize(country_request.json()["results"])

country_df.head(1)

,id,text
0,all,All


In [12]:
# read json file into pandas data frame
com_request = requests.get(url=full_com_url)
com_request.encoding='utf-8-sig'
com_df = pd.json_normalize(com_request.json()["results"])

com_df

,id,text,parent
0,ALL,ALL - All HS commodities,#
1,TOTAL,TOTAL - Total of all HS commodities,#
2,AG2,AG2 - All 2-digit HS commodities,#
3,AG4,AG4 - All 4-digit HS commodities,#
4,AG6,AG6 - All 6-digit HS commodities,#
...,...,...,...
8262,970690,970690 - Antiques; of an age exceeding 100 yea...,9706
8263,99,99 - Commodities not specified according to kind,TOTAL
8264,9999,9999 - Commodities not specified according to ...,99
8265,999999,999999 - Commodities not specified according t...,9999


In [22]:
twocom_request = requests.get(url=two_com_url)
twocom_request.encoding='utf-8-sig'
twocom_df = pd.json_normalize(twocom_request.json())

twocom_df

,id,text,parent
0,01,01 - Animals; live,TOTAL
1,01,01 - Animals; live,TOTAL
2,02,02 - Meat and edible meat offal,TOTAL
3,03,"03 - Fish and crustaceans, molluscs and other ...",TOTAL
4,04,04 - Dairy produce; birds' eggs; natural honey...,TOTAL
...,...,...,...
93,94,"94 - Furniture; bedding, mattresses, mattress ...",TOTAL
94,95,"95 - Toys, games and sports requisites; parts ...",TOTAL
95,96,96 - Miscellaneous manufactured articles,TOTAL
96,97,97 - Works of art; collectors' pieces and anti...,TOTAL


In [31]:
# read two digit commodity codes into json file into pandas data frame
twocom_request = requests.get(url=two_com_url)
twocom_df = pd.read_json(twocom_request.text)
# make the id column contain two digit commodity codes not one digit
twocom_df['id'] = twocom_df['id'].apply(lambda x: '{0:0>2}'.format(x))
twocom_df

,id,text,parent
0,01,01 - Animals; live,TOTAL
1,01,01 - Animals; live,TOTAL
2,02,02 - Meat and edible meat offal,TOTAL
3,03,"03 - Fish and crustaceans, molluscs and other ...",TOTAL
4,04,04 - Dairy produce; birds' eggs; natural honey...,TOTAL
...,...,...,...
93,94,"94 - Furniture; bedding, mattresses, mattress ...",TOTAL
94,95,"95 - Toys, games and sports requisites; parts ...",TOTAL
95,96,96 - Miscellaneous manufactured articles,TOTAL
96,97,97 - Works of art; collectors' pieces and anti...,TOTAL


In [32]:
# print each value from text column  
for val in twocom_df.text: 
    print(val)

01 - Animals; live
01 - Animals; live
02 - Meat and edible meat offal
03 - Fish and crustaceans, molluscs and other aquatic invertebrates
04 - Dairy produce; birds' eggs; natural honey; edible products of animal origin, not elsewhere specified or included
05 - Animal originated products; not elsewhere specified or included
06 - Trees and other plants, live; bulbs, roots and the like; cut flowers and ornamental foliage
07 - Vegetables and certain roots and tubers; edible
08 - Fruit and nuts, edible; peel of citrus fruit or melons
09 - Coffee, tea, mate and spices
10 - Cereals
11 - Products of the milling industry; malt, starches, inulin, wheat gluten
12 - Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
13 - Lac; gums, resins and other vegetable saps and extracts
14 - Vegetable plaiting materials; vegetable products not elsewhere specified or included
15 - Animal, vegetable or microbial fats and oils and their cleav

In [35]:
# full commodity list text values that begin with 31
x  = com_df[com_df['text'].str.startswith('31')]
# export x to csv
x.to_csv('fertilizer.csv', index=False)

In [131]:
food_list = ["01", "02", "03", "04", "05", "07", "08", "09", "10", "11", "12", "13", "15", "16", "17", "19", "20"]
twocom_df['id'] = twocom_df['id'].astype(str)
# dataframe in case we want to work with this list of commodities later
food_df = twocom_df[twocom_df['id'].str.startswith(tuple(food_list))]
# this is what will be used for the URL later
food_string = "%2C".join(food_list)
food_string

'01%2C02%2C03%2C04%2C05%2C07%2C08%2C09%2C10%2C11%2C12%2C13%2C15%2C16%2C17%2C19%2C20'

In [53]:
country_code = "586" # pakistan country code
year_string = "2021%2C2020%2C2019%2C2018%2C2017"
year = "2021"

In [127]:

ie_url = f"http://comtrade.un.org/api/get?max=1999&type=C&freq=A&px=HS&ps={year}&r={country_code}&p=all&rg=all&cc={food_string}"
ie_url

'http://comtrade.un.org/api/get?max=1999&type=C&freq=A&px=HS&ps=2021&r=586&p=all&rg=all&cc=01%2C02%2C03%2C04%2C05%2C07%2C08%2C09%2C10%2C11%2C12%2C13%2C15%2C16%2C17%2C19%2C20'

In [178]:
color_dict = {
            "(?)":"white",
            "Animals; live":"#807E5B",
            "Meat and edible meat offal":"#C9938B",
            "Dairy produce; birds' eggs; natural honey; edible products of animal origin, not elsewhere specified or included":"#99B7CF",
            "Animal originated products; not elsewhere specified or included":"#7D7E7D",
            "Vegetables and certain roots and tubers; edible":"#57633C",
            "Fruits and nuts; edible":"#98A552",
            "Cereals":"#B99A7A",
            "Coffee, tea, mate and spices":"#5F4737",
            "Fruit and nuts, edible; peel of citrus fruit or melons":"#C66562",
            "Products of the milling industry; malt, starches, inulin, wheat gluten":"#8D8D8E",
            "Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder":"#F3E18F",
            "Lac; gums, resins and other vegetable saps and extracts":"#B5A6C8",
            "Animal or vegetable fats and oils and their cleavage products; prepared animal fats; animal or vegetable waxes":"#EAAC55",
            "Sugars and sugar confectionery":"#EBE5D9",
            "Preparations of cereals, flour, starch or milk; pastrycooks' products":"#662C69",
            "Fish and crustaceans, molluscs and other aquatic invertebrates":"#A3AAA5",
            "Meat, fish or crustaceans, molluscs or other aquatic invertebrates; preparations thereof":"#3B768C",
            "Preparations of vegetables, fruit, nuts or other parts of plants":"#E95F85"
            }

def get_comtrade_data(endpoint_url):
     request = requests.get(url=endpoint_url)
     print (request)
     length = len(request.json()["dataset"])
     print ("length is " + str(length))
     if length > 1998:
           print ("Data may be truncated in graphs due to API limit")
     return (pd.json_normalize(request.json()["dataset"]))

def make_treegraph (dataframe, Year, Type):
     fig = px.treemap(dataframe, path=[px.Constant("all"), "cmdDescE", 'ptTitle'], values='TradeValue', color='cmdDescE', color_discrete_map=color_dict)
     fig.update_layout(margin = dict(t=50, l=0, r=0, b=0))
     fig.update_layout(title_text=f'{Type} by Commodity and Partner in {Year}')
     # remove outline color
     fig.update_traces(marker_line_color='rgb(0,0,0)', marker_line_width=.1)
     return(fig)

def commodity_charts(country_code):
    year = "2021"
    multiyear_string = "2021%2C2020%2C2019%2C2018%2C2017"
    # this is a string of all the two digit commodity codes that I consider to be food
    food_code_string = "01%2C02%2C03%2C04%2C05%2C07%2C08%2C09%2C10%2C11%2C12%2C13%2C15%2C16%2C17%2C19%2C20"
    import_url = f"http://comtrade.un.org/api/get?max=1999&type=C&freq=A&px=HS&ps={year}&r={country_code}&p=all&rg=1&cc={food_code_string}"
    export_url = f"http://comtrade.un.org/api/get?max=1999&type=C&freq=A&px=HS&ps={year}&r={country_code}&p=all&rg=2&cc={food_code_string}"
    import_df = get_comtrade_data(import_url)
    import_df = import_df[import_df['ptTitle'].str.match('World') == False]
    export_df = get_comtrade_data(export_url)
    export_df = export_df[export_df['ptTitle'].str.match('World') == False]
    
    import_treegraph = make_treegraph(import_df, year, "Imports")
    export_treegraph = make_treegraph(export_df, year, "Exports")
    return import_treegraph.show(), export_treegraph.show()



In [177]:
commodity_charts("586")

<Response [200]>
length is 668
<Response [200]>
length is 994


(None, None)

In [128]:
z = get_comtrade_data(ie_url)
z
# z.to_csv('allscratch.csv', index=False)

<Response [200]>
length is 1691


,pfCode,yr,period,periodDesc,aggrLevel,IsLeaf,rgCode,rgDesc,rtCode,rtTitle,...,qtAltCode,qtAltDesc,TradeQuantity,AltQuantity,NetWeight,GrossWeight,TradeValue,CIFValue,FOBValue,estCode
0,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,NaN,None,28524264,None,None,0
1,H5,2021,2021,2021,2,0,2,Export,586,Pakistan,...,None,,0,None,NaN,None,8918871,None,None,0
2,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,3129633,None,None,0
3,H5,2021,2021,2021,2,0,2,Export,586,Pakistan,...,None,,0,None,0.0,None,339933606,None,None,0
4,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,86904406,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,H5,2021,2021,2021,2,0,2,Export,586,Pakistan,...,None,,0,None,0.0,None,3785462,None,None,0
1687,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,914,None,None,0
1688,H5,2021,2021,2021,2,0,2,Export,586,Pakistan,...,None,,0,None,0.0,None,32467,None,None,0
1689,H5,2021,2021,2021,2,0,2,Export,586,Pakistan,...,None,,0,None,0.0,None,28791690,None,None,4


In [129]:
# return all columns whre rgDesc matches Import exactly
imports = z[z['rgDesc'].str.match('Import')]
imports = imports[imports['ptTitle'].str.match('World') == False]
imports
# export imports to csv
# imports.to_csv('importsscratch.csv', index=False)
# imports where ptTitle does not match World


,pfCode,yr,period,periodDesc,aggrLevel,IsLeaf,rgCode,rgDesc,rtCode,rtTitle,...,qtAltCode,qtAltDesc,TradeQuantity,AltQuantity,NetWeight,GrossWeight,TradeValue,CIFValue,FOBValue,estCode
46,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,468605,None,None,0
51,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,774,None,None,0
54,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,35794,None,None,0
56,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,4701985,None,None,0
59,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,139411685,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,13721,None,None,0
1675,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,58219,None,None,0
1678,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,941,None,None,0
1687,H5,2021,2021,2021,2,0,1,Import,586,Pakistan,...,None,,0,None,0.0,None,914,None,None,0


In [153]:
z
# return unique values of z[cmdDescE]
z['cmdDescE'].unique()

In [130]:
fig = px.treemap(imports, path=[px.Constant("all"), "cmdDescE", 'ptTitle'], 
                 values='TradeValue', color='cmdCode')
fig.update_layout(margin = dict(t=50, l=0, r=0, b=0))
fig.update_layout(title_text='Imports by Commodity and Partner')
# remove outline color
fig.update_traces(marker_line_color='rgb(0,0,0)', marker_line_width=.1)
fig.show()
